In [94]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

Scrap web data

In [95]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content)
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]
df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


Drop not assigned Borough

In [96]:
rows = df[(df['Borough'] == 'Not assigned')].index
df.drop(rows , inplace=True)

Add all neighborhoods of same boroughs together.
replace nan values of neighborhood with borough names first.

In [97]:
for index, row in df.iterrows():
    if pd.isnull(row['Neighborhood']):
        print("!")
        row['Neighborhood'] = row['Borough']

In [99]:
df = df.groupby('Postal code')['Neighborhood'].agg(','.join).reset_index()

replace nan values of neighborhood with borough names

In [100]:
for index, row in df.iterrows():
    if pd.isnull(row['Neighborhood']):
        row['Neighborhood'] = row['Borough']

In [101]:
df.shape

(103, 2)

In [85]:
data = pd.read_csv('Geospatial_Coordinates.csv')

In [88]:
data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [103]:
import geocoder # import geocoder
lat = []
lon = []

for index, row in df.iterrows():
    lat.append(data.iloc[0]['Latitude'])
    lon.append(data.loc[index]['Longitude'])

In [108]:
df['Latitude'] = lat
df['Longitude'] = lon
df

,Postal code,Neighborhood,Latitude,Longitude
0,M1B,Malvern / Rouge,43.806686,-79.194353
1,M1C,Rouge Hill / Port Union / Highland Creek,43.806686,-79.160497
2,M1E,Guildwood / Morningside / West Hill,43.806686,-79.188711
3,M1G,Woburn,43.806686,-79.216917
4,M1H,Cedarbrae,43.806686,-79.239476
...,...,...,...,...
98,M9N,Weston,43.806686,-79.518188
99,M9P,Westmount,43.806686,-79.532242
100,M9R,Kingsview Village / St. Phillips / Martin Grov...,43.806686,-79.554724
101,M9V,South Steeles / Silverstone / Humbergate / Jam...,43.806686,-79.588437


In [109]:
from sklearn.cluster import KMeans
kclusters = 5
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df[['Latitude','Longitude']])
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 3, 3, 3, 1, 3, 3], dtype=int32)